# November rain

November 2021 was the wettest November on record for Australia.

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from IPython.display import Image
from scipy.stats import genextreme as gev

from unseen import fileio
from unseen import general_utils
from unseen import indices
from unseen import time_utils

In [ ]:
# Optional parameters 
# (This cell is tagged "parameters")
dpi = 300

In [ ]:
# Required parameters
assert 'model_name' in locals(), "Must provide a model name"
assert 'min_lead' in locals(), "Must provide a minimum lead time"
assert os.path.isfile(bom_file), "Must provide an BoM data file (papermill option -p agcd_file [filepath])"
assert os.path.isfile(model_file), "Must provide an model data file (papermill option -p cafe_file [filepath])"
assert os.path.isfile(model_bc_file), "Must provide a model bias corrected data file (papermill option -p cafe_bc_file [filepath])"
assert os.path.isfile(similarity_bc_file), "Must provide an bias corrected similarity test file (papermill option -p similarity_bias_file [filepath])"
assert os.path.isfile(similarity_raw_file), "Must provide an raw data similarity test file (papermill option -p similarity_raw_file [filepath])"
assert os.path.isfile(independence_plot), "Must provide an independence test plot (papermill option -p independence_plot [filepath])"

## Observations

In [ ]:
Image(filename='/home/599/dbi599/nov-rain/analysis/pr_BoM_1900-2021_nov_aus-mean.png')

In [ ]:
bom_ds = fileio.open_dataset(bom_file)

In [ ]:
bom_ds

In [ ]:
years = bom_ds['time'].dt.year.values
bom_df = pd.DataFrame(index=years)
bom_df['pr'] = bom_ds['pr'].values

In [ ]:
mean_rainfall = bom_df.mean()
print(mean_rainfall)

In [ ]:
pr2021 = bom_ds['pr'].values.max()
print(pr2021)

In [ ]:
ranked_years = bom_df['pr'].sort_values(ascending=False)
ranked_years.head(n=10)

In [ ]:
bom_gev_shape, bom_gev_loc, bom_gev_scale = indices.fit_gev(bom_df['pr'].values)
print(f'Shape parameter: {bom_gev_shape:.2f}')
print(f'Location parameter: {bom_gev_loc:.2f}')
print(f'Scale parameter: {bom_gev_scale:.2f}')

In [ ]:
fig, ax = plt.subplots(figsize=[10, 8])
gev_xvals = np.arange(0, 120)
bom_df['pr'].plot.hist(bins=40, density=True, color='tab:green', alpha=0.5)
bom_gev_pdf = gev.pdf(gev_xvals, bom_gev_shape, bom_gev_loc, bom_gev_scale)
plt.plot(gev_xvals, bom_gev_pdf, color='tab:green', linewidth=4.0)
plt.show()

In [ ]:
for sample_size in [1000, 5000, 10000, 50000, 100000, 500000]:
    bom_gev_data = gev.rvs(bom_gev_shape, loc=bom_gev_loc, scale=bom_gev_scale, size=sample_size)
    bom_percentile, bom_return_period = general_utils.event_in_context(bom_gev_data, pr2021, 'above')
    print(f'Sample size: {sample_size}')
    print(f'{bom_percentile:.2f}% percentile')
    print(f'{bom_return_period:.0f} year return period\n')

Accordining to a GEV fitted to the observations, the event is in the 98th percentile with a 60-70 year return period.

## Model ensemble

In [ ]:
model_ds = fileio.open_dataset(model_file)
model_bc_ds = fileio.open_dataset(model_bc_file)

In [ ]:
model_ds

### Independence testing

In [ ]:
Image(filename=independence_plot)

In [ ]:
print(min_lead)

In [ ]:
model_da = model_ds['pr'].where(model_ds['lead_time'] >= min_lead)

### Bias correction and similarity testing

In [ ]:
fig = plt.figure(figsize=[10, 6])
model_da.plot.hist(bins=50, density=True, label=model_name, alpha=0.7)
model_bc_ds['pr'].plot.hist(bins=50, density=True, label=f'{model_name} bias corrected', facecolor='darkblue', alpha=0.7)
bom_ds['pr'].plot.hist(bins=50, density=True, label='BoM', facecolor='green', alpha=0.7)
plt.xlabel('precipitation (mm)')
plt.ylabel('probability')
plt.title(f'November rainfall for Australia ({model_name})')
plt.legend()
plt.savefig(f'/g/data/xv83/dbi599/nov-rain/figures/nov_precip_histogram_aus_{model_name}.png',
            bbox_inches='tight', facecolor='white', dpi=dpi)
plt.show()

In [ ]:
similarity_bias_ds = fileio.open_dataset(similarity_bc_file)
similarity_bias_ds['pval'].values

In [ ]:
similarity_raw_ds = fileio.open_dataset(similarity_raw_file)
similarity_raw_ds['pval'].values

These are the p-values for each lead time.

p > 0.05 means the null hypothesis (that the two samples are from the same population) can't be rejected.

### Exceedance curves

In [ ]:
def plot_exceedance(sample_ds, model_name):
    """Plot single exceedance curve"""
    
    sorted_data, exceedance_data = general_utils.exceedance_curve(sample_ds)
    
    fig = plt.figure(figsize=[8, 6])
    ax = fig.add_subplot()
    ax.plot(sorted_data, exceedance_data)
    ax.set_title(f'November rainfall for Australia ({model_name})')
    ax.set_ylabel('likelihood of exceedance (%)')
    ax.set_xlabel('monthly precipitation (mm)')
    ax.axvline(pr2021, color='0.5', linestyle='--')
    outfile = f'/g/data/xv83/dbi599/nov-rain/figures/nov_precip_exceedence_aus_{model_name}.png'
    plt.savefig(outfile, bbox_inches='tight', facecolor='white', dpi=dpi)
    print(outfile)

In [ ]:
def plot_exceedance_by_decade(sample_ds, model_name):
    """Plot exceedance curve by decade"""

    fig = plt.figure(figsize=[8, 6])
    ax = fig.add_subplot()
    start_years = [1960, 1970, 1980, 1990, 2000, 2010]
    colors = iter(plt.cm.hot_r(np.linspace(0.3, 1, len(start_years))))

    for start_year in start_years:
        end_year = start_year + 9
        start_date = f'{start_year}-01-01'
        end_date = f'{end_year}-12-31'
        ds_selection = time_utils.select_time_period(sample_ds, [start_date, end_date])
        ds_selection = ds_selection.dropna('sample')
        sorted_data, exceedance_data = general_utils.exceedance_curve(ds_selection)
        n_years = len(sorted_data)
        label = f'{start_year}-{end_year} ({n_years} samples)'
        color = next(colors)
        ax.plot(sorted_data, exceedance_data, label=label, color=color)
    
        print(f'{start_year}-{end_year}')
        percentile, return_period = general_utils.event_in_context(ds_selection.values, pr2021, 'above')
        print(f'{percentile:.2f}% percentile')
        print(f'{return_period:.0f} year return period\n')

    ax.set_title(f'November rainfall for Australia ({model_name})')
    ax.set_ylabel('likelihood of exceedance (%)')
    ax.set_xlabel('monthly precipitation (mm)')
    ax.legend()
    ax.axvline(pr2021, color='0.5', linestyle='--')
    outfile = f'/g/data/xv83/dbi599/nov-rain/figures/nov_precip_exceedence_aus_{model_name}_by-decade.png'
    plt.savefig(outfile, bbox_inches='tight', facecolor='white', dpi=dpi)
    print(outfile)

#### Bias corrected data

In [ ]:
model_bc_da_stacked = model_bc_ds.dropna('lead_time')['pr'].stack({'sample': ['ensemble', 'init_date', 'lead_time']})

In [ ]:
model_bc_da_stacked.shape

In [ ]:
plot_exceedance(model_bc_da_stacked, model_name)

In [ ]:
plot_exceedance_by_decade(model_bc_da_stacked, model_name)

In [ ]:
percentile_bc, return_period_bc = general_utils.event_in_context(model_bc_da_stacked.values, pr2021, 'above')
print('BIAS CORRECTED DATA')
print(f'{percentile_bc:.2f}% percentile')
print(f'{return_period_bc:.0f} year return period')

#### Raw data

In [ ]:
model_da_stacked = model_da.dropna('lead_time').stack({'sample': ['ensemble', 'init_date', 'lead_time']})

In [ ]:
model_da_stacked.shape

In [ ]:
plot_exceedance(model_da_stacked, model_name)

In [ ]:
percentile, return_period = general_utils.event_in_context(model_da_stacked.values, pr2021, 'above')
print('RAW DATA')
print(f'{percentile:.2f}% percentile')
print(f'{return_period:.0f} year return period')